# The effect of a Universal Child Benefit - González (2013) revisited

## A Project by Antonia Entorf & Marc Lipfert

#### Overview

This notebook replicates the main results presented in the following article:   

González, L. (2013): [The Effect of a Universal Child Benefit on Conceptions, Abortions, and Early Maternal Labor Supply](https://www.aeaweb.org/articles?id=10.1257/pol.5.3.160). American Economic Journal: Economic Policy 5(3): 160–188.

In that paper, the author investigates the effect of a universal child benefit on fertility, household expenditure patterns and maternal labour supply. In particular, she is able to exploit the introduction of such a policy that took place in Spain in 2007 by utilising a Regression Discontinuity Design.

Apart from replicating major findings presented by González, our aim is to enrich her analysis with respect to three different aspects. First, we will examine whether accounting for autocorrelation is necessary in the given context. Secondly, since the benefit was suspended in the aftermath of the financial crisis, we will exploit this fact and apply the identical research design to the abolishment of the policy in order to study the effect on conceptions. Thirdly, we will investigate threats to validity using simulations as well as a placebo test.




### Introduction